<a href="https://colab.research.google.com/github/2019mohamed/ArabDialectClassification/blob/master/SVM_TruncatedSVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/two_clo_dialect_dataset.csv')
df

,new_text,label
0,بالنهايه ينتفض يغير,0
1,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...,0
2,مبين كلامه خليجي,0
3,يسلملي مرورك وروحك الحلوه,0
4,وين الغيبه اخ محمد,0
...,...,...
458192,مبسوطين منك اللي باسطانا,17
458193,والله ماينده ابش يختي,17
458194,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي...,17
458195,الله يبارك وبالعافيه,17


In [ ]:
corpus = df['new_text'].astype('str').tolist()
labels = df['label'].tolist()

In [ ]:
freqs = df['label'].value_counts()
print(freqs)
print('-----------------------')

start = [0 for _ in range(18)]
start[0] = 0
for c in range(1,18):
    start[c] += freqs[c-1] + start[c-1]  
print(start)

indexes = []# (mini , maxi+1) for all classes
for c in range(18):
    indexes.append(  (start[c] , start[c] + freqs[c]) )
   
print(indexes)


for c in range(18):
    print(c,' ','-------------------')
    print(df.iloc[indexes[c][0]:indexes[c][1] , 1])

11    57636
3     43742
13    42109
1     36499
2     31069
6     27921
12    27617
8     26832
16    26296
17    26292
14    19116
4     16242
10    16183
0     15497
15    14434
7     11539
9      9927
5      9246
Name: label, dtype: int64
-----------------------
[0, 15497, 51996, 83065, 126807, 143049, 152295, 180216, 191755, 218587, 228514, 244697, 302333, 329950, 372059, 391175, 405609, 431905]
[(0, 15497), (15497, 51996), (51996, 83065), (83065, 126807), (126807, 143049), (143049, 152295), (152295, 180216), (180216, 191755), (191755, 218587), (218587, 228514), (228514, 244697), (244697, 302333), (302333, 329950), (329950, 372059), (372059, 391175), (391175, 405609), (405609, 431905), (431905, 458197)]
0   -------------------
0        0
1        0
2        0
3        0
4        0
        ..
15492    0
15493    0
15494    0
15495    0
15496    0
Name: label, Length: 15497, dtype: int64
1   -------------------
15497    1
15498    1
15499    1
15500    1
15501    1
        ..
51991  

In [ ]:
import numpy as np

In [ ]:
def sample_data (data , label ,intervals, r = 0.2):
  data = np.array(data)
  label = np.array(label)
  ret = np.array([])
  retl = np.array([])
  for i in range(18):
    l = intervals[i][0]
    h = intervals[i][1]
    index = np.random.choice(range(l,h), size = int((h-l) * r) , replace = False )
    ret = np.concatenate((ret , data[index]), axis = 0)
    retl = np.concatenate((retl , label[index]), axis = 0)

  return ret,retl

In [ ]:
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD

In [ ]:
vec = pickle.load(open('/content/drive/MyDrive/vectorizer.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
sample_corpus, sample_label = sample_data(corpus , labels, indexes , 0.1)

In [ ]:
X = vec.transform(sample_corpus)
X.shape

(45810, 423285)

In [ ]:
reducer = TruncatedSVD(n_components= 1000)
Z = reducer.fit_transform(X)
#plt.scatter(Z[:,0] , Z[:,1] , c = sample_label)
#plt.show()

In [ ]:
#pickle.dump(reducer, open('reducer.pkl','wb'))

In [ ]:
Z.shape

(45810, 1000)

In [ ]:
permute = np.random.permutation(Z.shape[0])
Z = Z[permute]
sample_label = sample_label[permute]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Z,sample_label, test_size=0.33, random_state=42)

In [ ]:
X_train.shape,' ',y_train.shape

((30692, 1000), ' ', (30692,))

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
model1 = SVC(kernel = 'linear')

In [ ]:
model1.fit(X_train,y_train)

SVC(kernel='linear')

In [ ]:
predict1 = model1.predict(X_test)

In [ ]:
Acc = accuracy_score(y_test,predict1)
Acc

0.3319883582484456